In [1]:
import googlemaps
import pandas as pd
import keyring
import json
from tqdm import tqdm

In [2]:
API_KEY = keyring.get_password("goomaps", "yianzhang14")

In [3]:
gmaps = googlemaps.Client(key=API_KEY)

In [4]:
df = pd.read_csv("data/urgent_time_final.csv")

In [5]:
# removed most duplicates but somehow there is shared place ID with different location
df["place_id"].value_counts().to_dict()

{'ChIJReOQsUJDwokRalaIuBckL8w': 2,
 'ChIJucG-Te1cwokRgHAub0PMu-8': 1,
 'ChIJ9RiNOMFYwokRySKwWFYLFaE': 1,
 'ChIJDcUcCepbwokRHclI_QuVK3o': 1,
 'ChIJ2fxcY2ZbwokR1vtuQg56V-Y': 1,
 'ChIJqUQTptZEwokRpKoIlZOWu2c': 1,
 'ChIJgaKlBH30wokRBaySI8YUKio': 1,
 'ChIJh3bCDy9FwokR5uG_-IUgYVA': 1,
 'ChIJxTWexz32wokRREwh80qRNVA': 1,
 'ChIJu805_RX1wokR_3jX2qqY5Zk': 1,
 'ChIJ-8nc34NcwokRNInVdBNnzBg': 1,
 'ChIJYQQ2v95bwokRLHImR-DwekA': 1,
 'ChIJv2sBjK1LwokRD-tqC-mbiW8': 1,
 'ChIJQYLVN8dYwokR_Qw2rFRKDbw': 1,
 'ChIJzVZStvNZwokRQzYWRsbNpWc': 1,
 'ChIJWVHqUd_ywokRNimV63O1ksw': 1,
 'ChIJ54o8gCfzwokRfYbY7XMiTRY': 1,
 'ChIJOQVyomZbwokRNKbvDahn6Vw': 1,
 'ChIJeW9QaeBhwokRw3OtSdDWWlk': 1,
 'ChIJOUWCotlbwokRZaNC5f8pNts': 1,
 'ChIJNwisSxFawokRdik5x8Wwj-Y': 1,
 'ChIJcRBztP1awokRNRv-v9lNCw8': 1,
 'ChIJ6zHWBJpZwokRviQPgDcrzdg': 1,
 'ChIJE6xKDOBhwokR66zQ_rOl27Q': 1,
 'ChIJV_XgU7FbwokR90HNw6RzRW4': 1,
 'ChIJXe6ga1lFwokRLWouJat5GgI': 1,
 'ChIJ8bcxFTSNwokR18sKuyf_dKo': 1,
 'ChIJl8w96kdEwokRuWIfnGFWaU4': 1,
 'ChIJqQ68uxtcwokRAH

In [6]:
for i, row in df.iterrows():
    print(i)
    if pd.isna(row["place_id"]):
        continue
    
    place = gmaps.place(row["place_id"])["result"]
    res = row.copy()
    if "formatted_address" in place:
        res["gmaps_address"] = place["formatted_address"]
    
    if "name" in place:
        res["gmaps_name"] = place["name"]
        
    if "business_status" in place:
        res["gmaps_business_status"] = place["business_status"]
        
    if "geometry" in place:
        res["gmaps_lat"] = place["geometry"]["location"]["lat"]
        res["gmaps_lon"] = place["geometry"]["location"]["lng"]
    
    df.iloc[i] = res

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
def set_open(row, day, start_minute, end_minute):
    for i in range(start_minute, end_minute, 30):
        row[f"{day}_{i}"] = 1

def set_all_open(row):
    for i in range(7):
        for j in range(0, 1440, 30):
            row[f"{i}_{j}"] = 1
    return row

In [8]:
for i in range(7):
    for j in range(0, 1440, 30):
        df[f"{i}_{j}"] = 0

C:\Users\fishl\AppData\Local\Temp\ipykernel_11472\3046168497.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{i}_{j}"] = 0
C:\Users\fishl\AppData\Local\Temp\ipykernel_11472\3046168497.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{i}_{j}"] = 0
C:\Users\fishl\AppData\Local\Temp\ipykernel_11472\3046168497.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

In [9]:
for i, row in df.iterrows():
    if pd.isna(row["opening_hours"]):
        continue
    curr = json.loads(row["opening_hours"].replace("'", '"').replace("\n", "").replace("False", "false").replace("True", "true"))
    periods = curr["periods"]
    temp = row.copy()
    if len(periods) == 1:
        df.loc[i] = set_all_open(row)
        continue
    for period in periods:
        close = period["close"]
        open = period["open"]
        
        open_hour = int(open["time"][:2])
        open_minute = int(open["time"][-2:])
        close_hour = int(close["time"][:2])
        close_minute = int(close["time"][-2:])
        if close["day"] != open["day"]:
            close_hour = 24
        
        set_open(temp, open["day"], open_hour * 60 + open_minute, close_hour * 60 + close_minute)
        
    df.loc[i] = temp
            
        

In [10]:
df.to_csv("data/final.csv")